In [ ]:
import numpy as np
total_bit = int(input())
tot_siz = 1<<total_bit
print("Total Number: ", tot_siz)

10
Total Number:  1024


In [ ]:
def proc_inp(val):
    ret = np.zeros(total_bit)
    for i in range(total_bit):
        ret[i] = (val>>i)&1

    return ret

Dataset Prep

In [ ]:
fixed_bit = 7
inp = []; out = []
clss = set()

for i in range(0, tot_siz):
    val = 0
    for k in range(0, fixed_bit):
        if((i>>k)&1):
            val |=(1<<k)

    inp.append(i), out.append(val)
    clss.add(val)

print("Classes: ", clss)

Classes:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127}


Train Test Split

In [ ]:
import random

def train_test_split(ratio):
    global train_x, train_y, test_x, test_y, indices

    train_x = []; train_y = []
    test_x = []; test_y = []

    indices = []
    for i in range(0, tot_siz):
        indices.append(i)

    random.shuffle(indices)
    #print(indices)

    train_siz = int(ratio*tot_siz)
    for i in range(0, train_siz):
        train_x.append(inp[indices[i]])
        train_y.append(out[indices[i]])

    for i in range(train_siz, tot_siz):
        test_x.append(inp[indices[i]])
        test_y.append(out[indices[i]])

    print("Train Size: ", len(train_x), "Test Size: ", len(test_x))

In [ ]:
def train_test_example():
    print("TrainSet Example:")
    for i in range(5):
        print("Data: ", proc_inp(train_x[i]), "Label: ", train_y[i])

    print("")
    print("TestSet Example:")
    for i in range(5):
        print("Data: ", proc_inp(test_x[i]), "Label: ", test_y[i])

Array Preparation

In [ ]:
import numpy as np

def prep_array():
    global train_x, train_y, test_x, test_y, w1, dw1, w2, dw2, u2, du2, u3, du3, o2, o3, layer1, layer2, layer3

    train_x = np.array(train_x); train_y = np.array(train_y)
    test_x = np.array(test_x); test_y = np.array(test_y)

    layer1 = total_bit; layer2 = 20; layer3 = fixed_bit

    w1 = np.zeros((layer1, layer2))
    dw1 = np.zeros((layer1, layer2))

    w2 = np.zeros((layer2, layer3))
    dw2 = np.zeros((layer2, layer3))

    u2 = np.zeros(layer2)
    du2 = np.zeros(layer2)

    u3 = np.zeros(layer3)
    du3 = np.zeros(layer3)

    for i in range(layer1):
        for j in range(layer2):
            w1[i][j] = random.random()

    for i in range(layer2):
        u2[i] = 0.5
        for j in range(layer3):
            w2[i][j] = random.random()

    for i in range(layer3):
        u3[i] = 0.5

    o2 = np.zeros(layer2)
    o3 = np.zeros(layer3)

eta1 = 0.9
eta2 = 0.9
th = 0.5

In [ ]:
def print_weight_matrix():
    print("Weight Matrix 1:", w1)
    print('')
    print("Weight Matrix 2:", w2)

In [ ]:
import math
tot_mat = 0; tot_error = 0
err = np.zeros(5)

def train(inp, lab):
    global tot_mat, tot_error, err

    # Block 1
    net = np.zeros(layer2)
    activ = np.zeros(layer2)
    K = 0.15
    K2 = 0.15
    A = 1

    for j in range(layer2):
        for i in range(layer1):
            net[j] += w1[i][j]*inp[i]

        activ[j] = net[j] + u2[j]
        o2[j] = 1.0/(A + math.exp(-K*activ[j]))

    # Block 2
    net = np.zeros(layer3)
    activ = np.zeros(layer3)
    err = np.zeros(layer3)

    for k in range(layer3):
        for j in range(layer2):
            net[k] += w2[j][k]*o2[j]

        activ[k] = net[k] + u3[k]
        o3[k] = 1.0/(A + math.exp(-K*activ[k]))

    # Block 3
    val = 0
    targ_feat = proc_inp(lab)
    for k in range(layer3):
        err[k] = targ_feat[k] - o3[k]
        tot_error += (targ_feat[k] - o3[k])**2

        cur_bit = 0
        if o3[k] >= th:
            cur_bit = 1

        if cur_bit == 1:
            val |= (1<<k)

    ok = 0
    if val == lab:
        ok = 1

    # Block 4 (Back Prop Starts)
    for j in range(layer2):
        for k in range(layer3):
            dw2[j][k] = eta2*K2*err[k]*o2[j]*o3[k]*(1-o3[k])
            w2[j][k] += dw2[j][k]

    for k in range(layer3):
        du3[k] = eta2*K2*err[k]*o3[k]*(1-o3[k])
        u3[k] += du3[k]

    # Block 5
    for i in range(layer1):
        for j in range(layer2):
            sum = 0
            for k in range(layer3):
                sum += err[k]*w2[j][k]

            dw1[i][j] = eta1*K*inp[i]*o2[j]*(1-o2[j])*sum
            w1[i][j] += dw1[i][j]

            du2[j] = eta1*K*o2[j]*(1-o2[j])*sum
            u2[j] += du2[j]

    return ok

In [ ]:
def exec_train():
    global tot_error
    epoch = 5
    for ep in range(epoch):
        tot_mat = 0; tot_error = 0

        for i in range(train_x.shape[0]):
            feat = proc_inp(train_x[i])
            tot_mat += train(feat, train_y[i])

        print("\n\nEpoch: ", ep, "Accuracy: ", tot_mat/train_x.shape[0]*100.0, "Total Loss: ", tot_error*0.5)

        print("\nOaj: ", o2)
        print("\nOak: ", o3)
        print("\nErr: ", err)

        print("\nBack Prop:")
        print("\nUok: ", u3)
        print("\nUhj: ", u2)


In [ ]:
def final_weight_matrix():
    print("Final Weight Matrix 1:", w1)
    print('')
    print("Final Weight Matrix 2:", w2)

In [ ]:
def pred(inp, lab, flag=False):
    global tot_error

    # Block 1
    net = np.zeros(layer2)
    activ = np.zeros(layer2)
    K = 0.15
    K2 = 0.15
    A = 1

    for j in range(layer2):
        for i in range(layer1):
            net[j] += w1[i][j]*inp[i]

        activ[j] = net[j] + u2[j]
        o2[j] = 1.0/(A + math.exp(-K*activ[j]))

    # Block 2
    net = np.zeros(layer3)
    activ = np.zeros(layer3)
    err = np.zeros(layer3)

    for k in range(layer3):
        for j in range(layer2):
            net[k] += w2[j][k]*o2[j]

        activ[k] = net[k] + u3[k]
        o3[k] = 1.0/(A + math.exp(-K*activ[k]))

    # Block 3
    val = 0
    targ_feat = proc_inp(lab)
    for k in range(layer3):
        err[k] = targ_feat[k] - o3[k]
        tot_error += (targ_feat[k] - o3[k])**2

        cur_bit = 0
        if o3[k] >= th:
            cur_bit = 1

        if cur_bit == 1:
            val |= (1<<k)

    ok = 0
    if val == lab:
        ok = 1

    if flag==True:
        print('Data: ', inp, 'Actual Label: ', lab, 'Predicted Label: ', val)

    return ok

In [ ]:
def exec_test():
    tot_error = 0; tot_mat = 0

    for i in range(test_x.shape[0]):
        feat = proc_inp(test_x[i])
        tot_mat += pred(feat, test_y[i], i<5)

    print("Test Accuracy: ", tot_mat/test_x.shape[0]*100.0)

In [ ]:
print("Splitting Into 60-40 Ratio:\n")
train_test_split(0.6); train_test_example(); prep_array(); print_weight_matrix(); exec_train(); final_weight_matrix(); exec_test();

Splitting Into 60-40 Ratio:

Train Size:  614 Test Size:  410
TrainSet Example:
Data:  [1. 1. 0. 1. 1. 1. 0. 1. 0. 0.] Label:  59
Data:  [0. 0. 1. 0. 0. 1. 1. 1. 0. 1.] Label:  100
Data:  [1. 0. 0. 1. 0. 1. 1. 1. 1. 1.] Label:  105
Data:  [0. 1. 1. 1. 0. 0. 1. 0. 1. 0.] Label:  78
Data:  [0. 0. 1. 1. 1. 1. 1. 0. 1. 1.] Label:  124

TestSet Example:
Data:  [0. 1. 0. 0. 0. 0. 0. 0. 1. 1.] Label:  2
Data:  [0. 0. 0. 1. 0. 0. 1. 1. 0. 1.] Label:  72
Data:  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] Label:  8
Data:  [0. 1. 1. 1. 1. 0. 1. 0. 1. 0.] Label:  94
Data:  [1. 0. 0. 0. 1. 1. 0. 0. 1. 0.] Label:  49
Weight Matrix 1: [[0.76468448 0.80957076 0.65385707 0.12621218 0.35307649 0.96134213
  0.11815298 0.10445341 0.27226009 0.42841671 0.27947637 0.11378112
  0.9382655  0.40866044 0.50767889 0.35656804 0.54250781 0.79819339
  0.12376201 0.55646279]
 [0.40944531 0.07526081 0.54126656 0.24381342 0.61497852 0.97249183
  0.28678236 0.25287046 0.60432805 0.0270143  0.67683312 0.05328867
  0.34058935 0.3352

In [ ]:
print("Splitting Into 70-30 Ratio:\n")
train_test_split(0.7); train_test_example(); prep_array(); print_weight_matrix(); exec_train(); final_weight_matrix(); exec_test();

Splitting Into 70-30 Ratio:

Train Size:  716 Test Size:  308
TrainSet Example:
Data:  [1. 1. 0. 0. 1. 0. 1. 1. 0. 0.] Label:  83
Data:  [1. 1. 1. 0. 1. 0. 1. 1. 0. 0.] Label:  87
Data:  [1. 1. 1. 1. 0. 0. 1. 1. 1. 1.] Label:  79
Data:  [0. 1. 1. 1. 0. 1. 0. 1. 1. 0.] Label:  46
Data:  [0. 1. 1. 0. 0. 0. 1. 1. 1. 0.] Label:  70

TestSet Example:
Data:  [0. 0. 0. 0. 0. 0. 1. 0. 0. 1.] Label:  64
Data:  [1. 1. 1. 0. 0. 1. 0. 0. 1. 0.] Label:  39
Data:  [1. 1. 0. 0. 0. 1. 0. 0. 1. 0.] Label:  35
Data:  [1. 1. 1. 0. 1. 0. 0. 1. 1. 1.] Label:  23
Data:  [0. 1. 1. 1. 1. 0. 1. 1. 1. 0.] Label:  94
Weight Matrix 1: [[1.09488038e-02 8.33615620e-02 3.15861822e-01 1.60460668e-01
  4.25274334e-01 8.87578682e-01 4.46232161e-01 5.30654242e-01
  3.30916470e-02 8.65255157e-01 7.01240170e-01 2.91124715e-01
  2.49850421e-01 4.71662661e-01 4.04342768e-03 9.07472631e-01
  3.81953465e-01 5.28500794e-01 7.20918029e-01 8.94330890e-01]
 [2.59351839e-01 2.51614009e-01 4.96857307e-01 2.07348245e-01
  5.21277103

In [ ]:
print("Splitting Into 80-20 Ratio:\n")
train_test_split(0.8); train_test_example(); prep_array(); print_weight_matrix(); exec_train(); final_weight_matrix(); exec_test();

Splitting Into 80-20 Ratio:

Train Size:  819 Test Size:  205
TrainSet Example:
Data:  [0. 1. 0. 0. 1. 0. 0. 0. 0. 1.] Label:  18
Data:  [1. 0. 1. 1. 0. 1. 1. 1. 1. 1.] Label:  109
Data:  [1. 0. 0. 0. 1. 1. 1. 0. 1. 1.] Label:  113
Data:  [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.] Label:  32
Data:  [1. 0. 0. 1. 0. 0. 0. 1. 0. 0.] Label:  9

TestSet Example:
Data:  [0. 0. 1. 0. 1. 0. 0. 0. 1. 0.] Label:  20
Data:  [1. 0. 0. 1. 0. 0. 1. 0. 1. 0.] Label:  73
Data:  [0. 0. 1. 1. 0. 0. 1. 1. 1. 1.] Label:  76
Data:  [1. 1. 0. 1. 0. 1. 0. 0. 1. 1.] Label:  43
Data:  [0. 0. 1. 1. 1. 1. 1. 1. 0. 1.] Label:  124
Weight Matrix 1: [[3.86398409e-01 5.64966798e-01 6.62067747e-01 9.62829639e-01
  7.10175242e-01 6.51842329e-01 8.65049512e-01 6.53576966e-01
  9.84711171e-01 9.46463023e-04 5.00150797e-01 4.63345097e-01
  3.83276640e-01 8.51865393e-01 3.47081914e-01 9.77680202e-01
  8.23339985e-01 4.05636551e-01 3.16235917e-01 9.59416119e-01]
 [3.72956261e-01 9.80691540e-01 9.56713533e-01 3.76420321e-01
  3.696721